In [ ]:
# K-Nearest Neighbors Evaluation Notebook

# Import standard packages
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# Import pipeline
import sys
import os
sys.path.append(os.path.abspath('..'))  # add project root to path
from pipeline import get_train_val_test_scaled

# Load data
X_train_scaled, X_val_scaled, X_test_scaled, y_train, y_val, y_test = get_train_val_test_scaled()
